# Model Traning

In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("data/gemstone.csv")
df

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453
...,...,...,...,...,...,...,...,...,...,...,...
193568,193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67,1130
193569,193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47,2874
193570,193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62,3036
193571,193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81,681


In [4]:
df.drop(columns='id',axis=1,inplace=True)

In [5]:
## Independent and dependent features
x=df.drop(columns=['price'],axis=1)
y=df[['price']]

In [6]:
x

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [7]:
y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [8]:
# Segregating numerical and categorical variables
categorical_col=x.select_dtypes(include='object').columns
numerical_col=x.select_dtypes(exclude='object').columns


In [9]:
# Define the custom ranking for each ordinal variable (increasing order)
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [10]:
from sklearn.impute import SimpleImputer # HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [11]:
# Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)

# Categorical Pipeline
cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)

preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_col),
    ('cat_pipeline',cat_pipeline,categorical_col)
])

In [12]:
## Train test split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=42)

In [13]:
x_train=pd.DataFrame(preprocessor.fit_transform(x_train),columns=preprocessor.get_feature_names_out())
x_test=pd.DataFrame(preprocessor.fit_transform(x_test),columns=preprocessor.get_feature_names_out())

In [14]:
x_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.823144,-1.129988,-0.641897,-0.780451,-0.835103,-0.876024,0.874100,-0.936747,1.350746
1,0.945023,-1.777823,0.921902,1.073226,1.166389,0.946633,-1.137644,0.910853,0.684455
2,1.958484,0.165682,0.400636,1.703116,1.755063,1.742237,-0.131772,0.910853,0.018164
3,-0.995648,-0.574701,-0.641897,-1.122391,-1.161138,-1.165334,0.874100,-0.320880,2.017037
4,-0.995648,0.258230,0.400636,-1.176382,-1.152082,-1.136403,-1.137644,1.526720,-0.648127


c:\Users\ARISH\Desktop\Data Science\PW_Skills\ML projects\Diamod_Price_Prediction\venv\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest: MSE=371939.75, R2=0.9770
----------------------------------------------------------------------------------------------------


In [ ]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [ ]:
import numpy as np
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae,rmse,r2_square

In [ ]:
## Train multiple models
## Model Evaluation
models={
    'DTRegressor':DecisionTreeRegressor(),
    'RandomForest':RandomForestRegressor(),
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)

    # make prediction
    y_pred=model.predict(x_test)

    mae,rmse,r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print('rmse',rmse)
    print('mae',mae)
    print('R2 Score',r2_square*100)

    r2_list.append(r2_square)

    print('-'*100)
   

DTRegressor
Model Training Performance
rmse 860.9335644668136
mae 432.24141295862603
R2 Score 95.4130170543021
----------------------------------------------------------------------------------------------------


c:\Users\ARISH\Desktop\Data Science\PW_Skills\ML projects\Diamod_Price_Prediction\venv\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForest
Model Training Performance
rmse 609.9953957910911
mae 310.77040384569136
R2 Score 97.69728091613828
----------------------------------------------------------------------------------------------------
LinearRegression
Model Training Performance
rmse 1015.0853051770308
mae 678.6599280042763
R2 Score 93.62334489637519
----------------------------------------------------------------------------------------------------
Lasso
Model Training Performance
rmse 1015.0659567672999
mae 679.9362394438181
R2 Score 93.62358798325504
----------------------------------------------------------------------------------------------------
Ridge
Model Training Performance
rmse 1015.0875649237871
mae 678.693463514639
R2 Score 93.62331650537858
----------------------------------------------------------------------------------------------------
ElasticNet
Model Training Performance
rmse 1526.1838883233627
mae 1065.7688041796291
R2 Score 85.5854338619943
--------------------------------------------

In [ ]:
# Random Forest Givng best accuracy among all

[0.954130170543021,
 0.9769728091613828,
 0.9362334489637519,
 0.9362358798325504,
 0.9362331650537858,
 0.855854338619943]

In [17]:
# Hyperparameter tuning Random forest Regressor
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

random_search = RandomizedSearchCV(estimator=rf_regressor, param_distributions=param_distributions, n_iter=10, cv=5, scoring='neg_mean_squared_error')
random_search.fit(x_train, y_train)

print(random_search.best_params_)
print(random_search.best_score_)

c:\Users\ARISH\Desktop\Data Science\PW_Skills\ML projects\Diamod_Price_Prediction\venv\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\ARISH\Desktop\Data Science\PW_Skills\ML projects\Diamod_Price_Prediction\venv\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\ARISH\Desktop\Data Science\PW_Skills\ML projects\Diamod_Price_Prediction\venv\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\A

{'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': 15}
-351020.67208844336


In [21]:
y_pred=random_search.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"{name}: MSE={mse:.2f}, R2={r2:.4f}")

Random Forest: MSE=349059.65, R2=0.9784
